In [1]:
# !pip install datasets transformers
# !pip install datasets --upgrade (otherwise issues to load the bean dataset)
# installations
# !pip install matplotlib
# !pip install pillow
# !pip install tifffile

In [2]:
# Run once, unless you have any update for the data structure
# create files with 47 channels at folder csiseminar/project/dataset/47_channels/
# the output is relevant to check that the files are not only zeros (viewing them, it looks like 
# it is only grey
# %run preprocess_tif_data.ipynb

In [3]:
from transformers.utils.dummy_vision_objects import ImageGPTFeatureExtractor
import random
from PIL import ImageDraw, ImageFont, Image

/Users/eilaarich-landkof-stanford/miniconda3/envs/transformers/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [4]:
import os
path_to_non_progressors_images = "/Users/eilaarich-landkof-stanford/Documents/Code/csiseminar/project/dataset/47_channels/labels/nonprogressor/"
path_to_progressors_images = "/Users/eilaarich-landkof-stanford/Documents/Code/csiseminar/project/dataset/47_channels/labels/progressor/"
CHANNELS = 47


In [5]:
import os
%run prepare_dataset_from_image_files.ipynb
non_progressors_images_list = os.listdir(path_to_non_progressors_images)
progressors_images_list = os.listdir(path_to_progressors_images)
non_progressors_images_list = [os.path.join(path_to_non_progressors_images,x) for x in non_progressors_images_list]
progressors_images_list = [os.path.join(path_to_progressors_images,x) for x in progressors_images_list]


non_progressors_images_dict = load_47_channel_tif_to_dataset_object(non_progressors_images_list,"nonprogressor")
progressors_images_dict = load_47_channel_tif_to_dataset_object(progressors_images_list,"progressor")

Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shap

In [6]:
# progressors_images_dict

In [7]:
# Create a Dataset object using Dataset.from_dict
%run prepare_dataset_from_image_files.ipynb
dataset_prog = Dataset.from_dict(progressors_images_dict) #,features={"image": "image", "labels": class_label})
dataset_non_prog = Dataset.from_dict(non_progressors_images_dict) #,features={"image": "image", "labels": class_label})


In [8]:
dataset_non_prog

Dataset({
    features: ['image', 'labels'],
    num_rows: 44
})

In [9]:
type(dataset_non_prog)

datasets.arrow_dataset.Dataset

In [10]:
from datasets import Dataset, concatenate_datasets
from datasets import DatasetDict


# Concatenate the datasets
ds = concatenate_datasets([dataset_prog, dataset_non_prog])
# 90% train, 10% test + validation - CAN BE CHANGED
train_testvalid = ds.train_test_split(test_size=0.1)
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
train_test_valid_dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test':  test_valid['test'],
    'valid': test_valid['train']})

In [11]:
train_test_valid_dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'labels'],
        num_rows: 52
    })
    test: Dataset({
        features: ['image', 'labels'],
        num_rows: 3
    })
    valid: Dataset({
        features: ['image', 'labels'],
        num_rows: 3
    })
})

In [12]:
train_test_valid_dataset['train']

Dataset({
    features: ['image', 'labels'],
    num_rows: 52
})

In [15]:
# Loading the feature extractor 
from transformers import ViTFeatureExtractor

model_name_or_path = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)


In [14]:
feature_extractor

ViTFeatureExtractor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTFeatureExtractor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [15]:
# train_test_valid_dataset['train']['image']
# accessing one image

In [16]:
# Assuming you have loaded your train dataset as train_dataset
train_dataset = train_test_valid_dataset['train']
train_dataset

Dataset({
    features: ['image', 'labels'],
    num_rows: 52
})

In [17]:

# Choose an index to access a specific image
index = 0  # Change this to the desired index

In [ ]:
# Access the image data and label at the chosen index
# image_data = train_dataset['image'][index]


In [ ]:
# image_data

In [ ]:

labels = train_dataset['Labels'][index]
labels


In [ ]:


# Print information about the image
print("Image shape:", image_data.shape)
print("labels:", labels)


In [ ]:
type(feature_extractor)

In [45]:
# try the feature_extractor on this image
feature_extractor(image_data, return_tensors='pt')

In [ ]:
# feature_extractor(train_split[, return_tensors='pt')

In [ ]:
# def process_example(example):
#     inputs = feature_extractor(example['image'], return_tensors='pt')
#     inputs['labels'] = example['labels']
#     return inputs